In [3]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [50]:
def get_jobs(keyword, num_jobs, verbose=True):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    path = "./chromedriver"
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1920, 1080) #(1120, 1000)
    
    #driver.execute_script("document.body.style.zoom='25%'")                 locId=86&locT=N&locName=France
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locId=86&locT=N&locName=France&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=false&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    #driver.execute_script("document.body.style.zoom='0.5'")
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(1)

        #Test for the "Sign Up" prompt and get rid of it.
       
        try:
            driver.find_element_by_class_name("selected").click()
        except   NoSuchElementException:
            pass

        time.sleep(4)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
            print('X out worked')
        except NoSuchElementException:
            print('X out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.
        
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            job_button.location_once_scrolled_into_view
            job_button.click()  #You might
            time.sleep(2)
                
                #___________ code to kill the sign-up pop-up after it render on screen
                # if not found_popup:
            try:
                    driver.find_element_by_css_selector('[alt="Close"]').click()
                    # print("&&& line 89")
                    # found_popup = True
            except NoSuchElementException:
                    # print("&&& line 92")
                    pass
            '''
            try:   
                job_button.click()  #You might
            except:
                
                #___________ code to kill the sign-up pop-up after it render on screen
                # if not found_popup:
                try:
                    driver.find_element_by_css_selector('[alt="Close"]').click()
                    # print("&&& line 89")
                    # found_popup = True
                except NoSuchElementException:
                    # print("&&& line 92")
                    pass
            '''
            #time.sleep(1)
            #time.sleep(.1)

            
            #pass
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="css-87uc0g e1tk4kwz1"]').text  
                    location = driver.find_element_by_xpath('.//div[@class="css-56kyx5 e1tk4kwz5"]').text
                    job_title = driver.find_element_by_xpath('.//div[@class="css-1vg6q84 e1tk4kwz4"]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

          
            
            try:
                salary_estimate = driver.find_element_by_xpath('.//div[@class="css-1bluz6i e2u4hf13"]').text
                salary_estimate_ = driver.find_elements_by_xpath('.//span[@class="css-1d4p0fd e2u4hf18"]') #.text
                min_salary_estimate = salary_estimate_[0].text
                max_salary_estimate = salary_estimate_[1].text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                min_salary_estimate = -1
                max_salary_estimate = -1
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="css-1m5m32b e1tk4kwz2"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}-{}".format(min_salary_estimate, max_salary_estimate))
                print("Average Base Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                company = driver.find_elements_by_xpath('.//div[@class="d-flex justify-content-start css-rmzuhb e1pvx6aw0"]//span[@class="css-i9gxme e1pvx6aw2"]')
                lable = driver.find_elements_by_xpath('.//div[@class="d-flex justify-content-start css-rmzuhb e1pvx6aw0"]//span[@class="css-1taruhi e1pvx6aw1"]') #.text
                lable = [str(c.text) for c in lable]
                tap = {key: value.text for key, value in zip(lable, company)}
            
                size = tap['Size'] if 'Size' in tap else -1  
                founded = tap['Founded'] if 'Founded' in tap else -1  
                type_of_ownership = tap['Type'] if 'Type' in tap else -1  
                industry = tap['Industry'] if 'Industry' in tap else -1  
                sector = tap['Sector'] if 'Sector' in tap else -1  
                revenue = tap['Revenue'] if 'Revenue' in tap else -1  
                #headquarters =  -1 

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                #competitors = -1

                
            if verbose:
                #print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                # print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"jobTitle" : job_title,
            "avgSalaryEstimate" : salary_estimate,
            "minSalaryEstimate" : min_salary_estimate,
            "maxSalaryEstimate" : max_salary_estimate,
            "jobDescription" : job_description,
            "rating" : rating,
            "companyName" : company_name,
            "location" : location,
            "size" : size,
            "founded" : founded,
            "typeOfOwnership" : type_of_ownership,
            "industry" : industry,
            "sector" : sector,
            "revenue" : revenue})
            #"Competitors" : competitors
            #add job to jobs
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="css-114lpwu e1gri00l4"]//a').click()
        except ElementClickInterceptedException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [51]:
#This line will open a new chrome window and start the scraping.
start = time.time()
df = get_jobs('data scientist',30)
end = time.time()

X out failed
Progress: 0/30
Job Title: Data Scientist h/f
Salary Estimate: -1--1
Average Base Salary Estimate: -1
Job Description: Venez nous apporter votre nuance !
Leader français du prêt-à-porter féminin, avec plus de 500 magasins, Camaïeu est avant tout une formidable école du retail !
Rejoindre Camaïeu, c’est intégrer un groupe de plus de 2400 collaborateurs où le plaisir et la confiance s’expriment à chaque instant pour apporter le meilleur à la cliente.

Data Scientist h/f

Référence : DSMD/59/9038/08807
Lieu : 59 - Siège Roubaix
Contrat : CDI

Vos responsabilités



Votre expérience
Rating: 3.4
Company Name: Camaieu
3.4
Location: Roubaix
Size: 1001 to 5000 Employees
Founded: 1984
Type of Ownership: Company - Public
Industry: Department, Clothing, & Shoe Stores
Sector: Retail
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 1/30
Job Title: Data Scientist/Analyst (F/H)
Salary Estimate: -1--1
Average Base Salary Estimate: -1
Job Desc

Job Title: DATA SCIENTIST - PREDICTIVE ANALYTICS - H/F
Salary Estimate: -1--1
Average Base Salary Estimate: -1
Job Description: DATA SCIENTIST - PREDICTIVE ANALYTICS - H/F (NUMÉRO DE L'EMPLOI : !I_CAR_0096)

Data Scientist - Predictive Analytics - H/F
Apporter votre expertise à l’actuariat des différents pays de BNP Paribas Cardif !
#machinelearning #Analytics #IA #R&D
Concrètement votre quotidien ?
Votre mission sera d’apporter du Machine Learning pour l'actuariat et l'analytics pour optimiser la tarification, la gestion du risque et améliorer les produits et services de BNP Paribas Cardif.
Rating: 3.7
Company Name: BNP Paribas
3.7
Location: Nanterre
Size: 10000+ Employees
Founded: 2000
Type of Ownership: Company - Public
Industry: Banks & Credit Unions
Sector: Finance
Revenue: $10+ billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 11/30
Job Title: Data Scientist H/F - Yves Rocher
Salary Estimate: -1--1
Average Base Salary Estimate: -1
Job Description: Réf: 

Job Title: Data Scientist Débutant(e)
Salary Estimate: -1--1
Average Base Salary Estimate: -1
Job Description: Géoscience & Réservoir
PAU-CSTJF(FRA)
France
Description du poste
TotalEnergies est une compagnie multi-énergies mondiale de production et de fourniture d’énergies : pétrole et biocarburants, gaz naturel et gaz verts, renouvelables et électricité. Ses 105 000 collaborateurs s'engagent pour une énergie toujours plus abordable, propre, fiable et accessible au plus grand nombre. Présent dans plus de 130 pays, TotalEnergies inscrit le développement durable dans toutes ses dimensions au cœur de ses p
Rating: 4.0
Company Name: TotalEnergies
4.0
Location: Pau
Size: 10000+ Employees
Founded: 1924
Type of Ownership: Company - Public
Industry: Energy
Sector: Oil, Gas, Energy & Utilities
Revenue: $10+ billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 21/30
Job Title: Data Scientist Junior H/F
Salary Estimate: -1--1
Average Base Salary Estimate: -1
Job Descripti

In [54]:
end - start

93.00482845306396

In [52]:
df

,jobTitle,avgSalaryEstimate,minSalaryEstimate,maxSalaryEstimate,jobDescription,rating,companyName,location,size,founded,typeOfOwnership,industry,sector,revenue
0,Data Scientist h/f,-1,-1,-1,Venez nous apporter votre nuance !\nLeader fra...,3.4,Camaieu\n3.4,Roubaix,1001 to 5000 Employees,1984,Company - Public,"Department, Clothing, & Shoe Stores",Retail,Unknown / Non-Applicable
1,Data Scientist/Analyst (F/H),-1,-1,-1,Carnet de route\nData Scientist/Analyst depuis...,4.1,Novencia\n4.1,Paris,201 to 500 Employees,2000,Company - Private,Architectural & Engineering Services,Business Services,$25 to $50 million (USD)
2,DATA SCIENTIST H/F,-1,-1,-1,DATA SCIENTIST H/F (NUMÉRO DE L'EMPLOI : !I_PF...,3.7,BNP Paribas\n3.7,Levallois-Perret,10000+ Employees,2000,Company - Public,Banks & Credit Unions,Finance,$10+ billion (USD)
3,Data Scientist H/F,-1,-1,-1,Environnement\nFintech interne du groupe Crédi...,-1,CRAFT,Paris,-1,-1,-1,-1,-1,-1
4,DATA SCIENTIST - TRAITEMENT DE DOCUMENTS MULTI...,-1,-1,-1,Descriptif du poste:\nL’expertise technologiqu...,3.4,EURO-INFORMATION DEVELOPPEMENTS\n3.4,Fontenay-sous-Bois,1001 to 5000 Employees,1991,Subsidiary or Business Segment,IT Services,Information Technology,Unknown / Non-Applicable
5,Data scientist-(H/F),-1,-1,-1,Vos missions au quotidien\nVous savez « faire ...,3.6,Société Générale\n3.6,Courbevoie,10000+ Employees,1864,Company - Public,Investment Banking & Asset Management,Finance,$10+ billion (USD)
6,Junior Data Scientist H/F,-1,-1,-1,"Au sein de la nouvelle Direction Data, chargée...",3.3,JCDecaux\n3.3,Neuilly-sur-Seine,10000+ Employees,1964,Company - Public,Accounting,Accounting & Legal,$500 million to $1 billion (USD)
7,Data Scientist Recherche Marchés H/F,-1,-1,-1,Description du poste\nLes Marchés de Capitaux ...,3.7,Crédit Agricole CIB\n3.7,Montrouge,5001 to 10000 Employees,1885,Company - Public,Investment Banking & Asset Management,Finance,$10+ billion (USD)
8,Ingénieur Data Scientist H/F,-1,-1,-1,Dans le cadre du développement des activités d...,3.0,AKKA TECHNOLOGIES\n3.0,Paris,10000+ Employees,1984,Company - Private,Research & Development,Business Services,$2 to $5 billion (USD)
9,Data scientist F/H,-1,-1,-1,CDI\nTemps plein\n27-31 avenue du Général Lecl...,4.3,Bpifrance\n4.3,Maisons-Alfort,1001 to 5000 Employees,2012,Company - Private,Banks & Credit Unions,Finance,$100 to $500 million (USD)


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   jobTitle           30 non-null     object
 1   avgSalaryEstimate  30 non-null     int64 
 2   minSalaryEstimate  30 non-null     int64 
 3   maxSalaryEstimate  30 non-null     int64 
 4   jobDescription     30 non-null     object
 5   rating             30 non-null     object
 6   companyName        30 non-null     object
 7   location           30 non-null     object
 8   size               30 non-null     object
 9   founded            30 non-null     object
 10  typeOfOwnership    30 non-null     object
 11  industry           30 non-null     object
 12  sector             30 non-null     object
 13  revenue            30 non-null     object
dtypes: int64(3), object(11)
memory usage: 3.4+ KB


In [41]:
job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")

In [42]:
job_buttons[0].get_attribute("data-id")

'4141430293'

In [43]:
job_buttons[0].get_attribute("data-is-easy-apply")

'true'

In [ ]:
attributes = ["data-id","data-is-easy-apply", 'data-normalize-job-title', 'data-job-loc-id', 'data-job-loc-type']

In [32]:
job_buttons[5].location_once_scrolled_into_view
salary_estimate = driver.find_elements_by_xpath('.//span[@class="css-1d4p0fd e2u4hf18"]') #.text
min_salary_estimate = salary_estimate[0].text
max_salary_estimate = salary_estimate[1].text

In [24]:
min_salary_estimate

'$96K '

In [25]:
max_salary_estimate

'$179K'